In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"james4510","key":"ecadc1536334386aec3711313d02eba7"}'}

In [2]:
!mkdir -p ~/.kaggle # kaggle이라는 숨겨진 폴더 생성
!cp kaggle.json ~/.kaggle/ # 폴더 안에 아까 다운 받은 json 파일 복사
!chmod 600 ~/.kaggle/kaggle.json # json 파일 권한 변경

In [3]:
#!/bin/bash
!kaggle datasets download zygmunt/goodbooks-10k
!kaggle datasets download arashnic/book-recommendation-dataset

Dataset URL: https://www.kaggle.com/datasets/zygmunt/goodbooks-10k
License(s): CC-BY-SA-4.0
 43% 5.00M/11.6M [00:00<00:00, 30.2MB/s]
100% 11.6M/11.6M [00:00<00:00, 49.5MB/s]
Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 90% 22.0M/24.3M [00:01<00:00, 22.5MB/s]
100% 24.3M/24.3M [00:01<00:00, 24.4MB/s]


In [4]:
from zipfile import ZipFile
file_name = "/content/goodbooks-10k.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [5]:
from zipfile import ZipFile
file_name = "/content/book-recommendation-dataset.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [6]:
import pandas as pd

# 데이터 읽기 - Goodbooks-10k dataset
gb_books = pd.read_csv("/content/books.csv")
gb_book_tags = pd.read_csv("/content/book_tags.csv")
gb_ratings = pd.read_csv("/content/ratings.csv")
gb_tags = pd.read_csv("/content/tags.csv")
gb_to_read = pd.read_csv("/content/to_read.csv")

# Book Recommendation dataset
br_books = pd.read_csv("/content/Books.csv")
br_ratings = pd.read_csv("/content/Ratings.csv")
br_users = pd.read_csv("/content/Users.csv")

<ipython-input-6-6e51e6ca2144>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  br_books = pd.read_csv("/content/Books.csv")


In [7]:
# Goodbooks-10k dataset preprocessing
# 필요한 열만 남기기
# books 데이터셋에서 제목, 저자, 출판연도, ISBN 정보만 유지
gb_books = gb_books[['book_id', 'isbn', 'authors', 'original_publication_year', 'original_title']]
gb_ratings = gb_ratings[['user_id', 'book_id', 'rating']]

# books와 ratings 데이터셋을 book_id를 기준으로 결합
gb_merged_data = gb_ratings.merge(gb_books, on='book_id', how='inner')

# 결측치 처리
gb_merged_data['original_title'].fillna('Unknown Title', inplace=True)

<ipython-input-7-8b81dec6aab7>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gb_merged_data['original_title'].fillna('Unknown Title', inplace=True)


In [8]:
# Book Recommendation dataset preprocessing
# 결측치 처리 및 기본 정리
columns_to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']
br_books.drop(columns=columns_to_drop, inplace=True)

# Remove missing values and convert all text data to lowercase for each dataset
br_books = br_books.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)
br_ratings = br_ratings.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)
br_users = br_users.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Merging the Ratings and Books datasets on the common ISBN column
br_merged_df = pd.merge(br_ratings, br_books, on='ISBN')

# Merging the result with the Users dataset on the common User-ID column
br_merged_df = pd.merge(br_merged_df, br_users, on='User-ID')

# Removing any outlier
br_delete_outliers = br_merged_df[br_merged_df['Book-Rating'] != 0]

<ipython-input-8-bc6a56b7d6c0>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  br_books = br_books.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)
<ipython-input-8-bc6a56b7d6c0>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  br_ratings = br_ratings.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)
<ipython-input-8-bc6a56b7d6c0>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  br_users = br_users.dropna().applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [9]:
br_delete_outliers.info(), gb_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273291 entries, 1 to 762993
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              273291 non-null  int64  
 1   ISBN                 273291 non-null  object 
 2   Book-Rating          273291 non-null  int64  
 3   Book-Title           273291 non-null  object 
 4   Book-Author          273291 non-null  object 
 5   Year-Of-Publication  273291 non-null  object 
 6   Publisher            273291 non-null  object 
 7   Location             273291 non-null  object 
 8   Age                  273291 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 20.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79701 entries, 0 to 79700
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    79701 non-null  int

(None, None)

In [10]:
# For the `goodbooks_10k.csv` dataset, let's filter based on ratings
# Count the number of ratings for each book and filter out books with fewer than 10 ratings
gb_goodbooks_filtered = gb_merged_data.groupby('book_id').filter(lambda x: len(x) >= 10)

# Then, filter out books rated by fewer than 5 unique users
gb_goodbooks_filtered = gb_goodbooks_filtered.groupby('book_id').filter(lambda x: x['user_id'].nunique() >= 5)

In [11]:
# For the `book_recommendation.csv` dataset
# Count the number of ratings for each book and filter out books with fewer than 10 ratings
br_recommendation_filtered = br_delete_outliers.groupby('ISBN').filter(lambda x: len(x) >= 10)

# Then, filter out books rated by fewer than 5 unique users
br_recommendation_filtered = br_recommendation_filtered.groupby('ISBN').filter(lambda x: x['User-ID'].nunique() >= 5)

In [12]:
# Standardize the 'user_id' and 'ISBN' columns across both dataframes

# Rename columns to ensure consistency in naming
gb_goodbooks_df = gb_goodbooks_filtered.rename(columns={'user_id': 'User_ID', 'isbn': 'ISBN', 'rating': 'Rating'})
br_recommendation_df = br_recommendation_filtered.rename(columns={'User-ID': 'User_ID', 'Book-Rating': 'Rating'})

In [13]:
# Normalize ISBN and User_ID to string format for consistency
gb_goodbooks_df['ISBN'] = gb_goodbooks_df['ISBN'].astype(str)
gb_goodbooks_df['User_ID'] = gb_goodbooks_df['User_ID'].astype(str)
br_recommendation_df['ISBN'] = br_recommendation_df['ISBN'].astype(str)
br_recommendation_df['User_ID'] = br_recommendation_df['User_ID'].astype(str)

In [14]:
goodbooks_isbn_map = gb_goodbooks_df.groupby('original_title')['ISBN'].first().to_dict()
recommendation_isbn_map = br_recommendation_df.groupby('Book-Title')['ISBN'].first().to_dict()

In [15]:
# Function to get the ISBN for the same title across datasets
def get_unified_isbn(title, isbn_map):
    return isbn_map.get(title, None)

# Update ISBN in goodbooks dataset using its original title
gb_goodbooks_df['ISBN'] = gb_goodbooks_df['original_title'].apply(
    lambda title: get_unified_isbn(title, goodbooks_isbn_map)
)

# Update ISBN in recommendation dataset using its book title
br_recommendation_df['ISBN'] = br_recommendation_df['Book-Title'].apply(
    lambda title: get_unified_isbn(title, recommendation_isbn_map)
)

In [16]:
# Adjust ISBN to 10 characters with leading zeros if necessary
gb_goodbooks_df['ISBN'] = gb_goodbooks_df['ISBN'].astype(str).str.zfill(10)
br_recommendation_df['ISBN'] = br_recommendation_df['ISBN'].astype(str).str.zfill(10)

In [17]:
# 하나의 책에 대해서만 평점을 매긴 유저 제거
# Filter users who have rated only one book in each dataset

# In goodbooks dataset
goodbooks_filtered = gb_goodbooks_df.groupby('User_ID').filter(lambda x: len(x) > 1)

# In recommendation dataset
recommendation_filtered = br_recommendation_df.groupby('User_ID').filter(lambda x: len(x) > 1)

In [18]:
# 두 개의 데이터셋에서 공통의 책을 제외한 나머지 책 제거
# Find common books between the two datasets based on the 'ISBN' column
common_books_isbn = set(goodbooks_filtered['ISBN']).intersection(set(recommendation_filtered['ISBN']))

# Filter out common books in both datasets
goodbooks_filtered = goodbooks_filtered[~goodbooks_filtered['ISBN'].isin(common_books_isbn)]
recommendation_filtered = recommendation_filtered[~recommendation_filtered['ISBN'].isin(common_books_isbn)]

# Save the filtered datasets to new CSV files
goodbooks_filtered.to_csv('/content/goodbooks_exclusive.csv', index=False, encoding="utf-8-sig")
recommendation_filtered.to_csv('/content/recommendation_exclusive.csv', index=False, encoding="utf-8-sig")